In [1]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [2]:
# Load the second NetCDF file with lat and lon coordinates
ds = xr.open_dataset('../output/mt+dis.nc')
#ds['Band1'] = ds['Band1'].fillna(0)
#ds['Band1'] = 1- ds['Band1']
#band1 = ds['Band1']

KeyError: 'Band1'

In [ ]:
# Load the NetCDF file with 'Band1' data variable
dataset = ds
band1 = dataset['Band1']

# Create a custom colormap with lightgrey for 0 and None (transparent) for 1
cmap = plt.cm.colors.ListedColormap(['None', 'lightgrey'])

# Create a figure and subplot with PlateCarree projection
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()}, figsize=(12, 6))

# Plot the 'Band1' data variable with the custom colormap
band1.plot.imshow(ax=ax, cmap=cmap, add_colorbar=False)

# Set the plot extent to cover the entire world
ax.set_global()

# Add coastlines and gridlines
ax.coastlines()
ax.gridlines()

# Set plot title and labels
ax.set_title('Band1 Data on World Map')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Show the plot
plt.show()

In [ ]:
# Load the NetCDF file with 'Band1' data variable
dataset = ds
band1 = dataset['Band1']

# Create a custom colormap with lightgrey for 0 and None (transparent) for 1
cmap = plt.cm.colors.ListedColormap(['None', 'lightgrey'])

# Create a figure and subplot with PlateCarree projection
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()}, figsize=(12, 6))

# Plot the 'Band1' data variable with the custom colormap
band1.plot.imshow(ax=ax, cmap=cmap, add_colorbar=False)

# Set the plot extent to cover the entire world
ax.set_global()

# Add coastlines and gridlines
ax.coastlines()
ax.gridlines()

# Set plot title and labels
ax.set_title('Band1 Data on World Map')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Show the plot
plt.show()

In [ ]:
# Create a figure and subplot
fig, ax = plt.subplots(figsize=(12, 6), subplot_kw={'projection': ccrs.PlateCarree()})

# Add land and ocean background
ax.add_feature(cfeature.LAND, facecolor='lightgray')

# Iterate over each threshold and plot filled contour regions
for i in range(len(thresholds)):
    threshold = thresholds[i]
    mask = year_dataset < threshold
    
    # Apply consecutive days condition
    consecutive_days = np.zeros_like(mask, dtype=int)
    consecutive_days[0] = mask[0].astype(int)
    
    for t in range(1, len(mask)):
        consecutive_days[t] = mask[t].astype(int) * (consecutive_days[t-1] + 1)
    
    mask_consecutive = consecutive_days >= window_size
    
    mask_2d = np.any(mask_consecutive, axis=0)
    
    ax.contourf(lon, lat, mask_2d, levels=[0.5, 1], colors=[plt.cm.Blues(i / (len(thresholds) - 1)), 'none'], alpha=0.5, hatches=[None, '////'], transform=ccrs.PlateCarree(), zorder=1)

# Add country outlines
ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='black')

# Create a custom colormap with lightgrey for 0 and None (transparent) for 1
cmap = plt.cm.colors.ListedColormap(['None', 'lightgrey'])
# Plot the 'Band1' data variable with the custom colormap
band1.plot.imshow(ax=ax, cmap=cmap, add_colorbar=False, zorder=3)

# Plot water-stressed regions as a background
#ax.add_geometries(df['geometry'], crs=ccrs.PlateCarree(), facecolor='red', zorder=2)

# Plot the RiverAtlas data
#df.plot(color='red',ax=ax, zorder=2)

# Set the plot extent to cover the entire world
ax.set_global()
ax.add_feature(cfeature.OCEAN, facecolor='lightblue', zorder=4)

# Set plot title and labels
ax.set_title('Mountain regions with suitable weather and discharge for artificial glaciers construction')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Add map features
ax.coastlines()
ax.gridlines()

# Create a custom legend
threshold_labels = [str(threshold) + ' °C' for threshold in thresholds]
legend_handles = [mpatches.Rectangle((0, 0), 1, 1, fc=plt.cm.Blues(i / (len(thresholds) - 1)), alpha=0.5) for i in range(len(thresholds))]

ax.legend(legend_handles, threshold_labels + ['No discharge'], title='Thresholds', loc='lower left')

# Save the figure
plt.savefig("../figs/mt_dis_wetbulb_layer.png", bbox_inches="tight", dpi=300)

# Show the plot
plt.show()